In [5]:
%load_ext autoreload
%autoreload 1
%aimport prediction_models

import numpy as np 
import pandas as pd 
import json
from pandas.io.json import json_normalize
import nltk
import copy
import random
from collections import Counter
import operator
import math

from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSequenceClassification, pipeline, T5ForConditionalGeneration, T5Tokenizer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion")

/home/ICT2000/chawla/anaconda3/envs/base_acii/lib/python3.7/site-packages/transformers/models/auto/modeling_auto.py:1010: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [34]:
text = 'i have a feeling i kinda lost my best friend'
print("text", text)

input_ids = tokenizer.encode(text + '</s>', return_tensors='pt')
print("input_ids", input_ids)

output = model.generate(input_ids=input_ids, max_length=2)
print("output", output)

dec = [tokenizer.decode(ids) for ids in output]
print("dec", dec)

label = dec[0].split()[-1]
print("label", label)

text i have a feeling i kinda lost my best friend
input_ids tensor([[   3,   23,   43,    3,    9, 1829,    3,   23,  773,    9, 1513,   82,
          200, 1565,    1]])
output tensor([[    0, 24784]])
dec ['<pad> sadness']
label sadness


In [119]:
text = 'i am very happy'
print("text", text)

input_ids = tokenizer.encode(text, return_tensors='pt')
print("input_ids", input_ids)

#outputs = model(input_ids=input_ids, decoder_input_ids=input_ids, return_dict=True, output_attentions=True, output_hidden_states=True)

output = model.generate(input_ids=input_ids, max_length=2, output_scores=True, output_hidden_states=True, output_attentions=True, return_dict_in_generate=True)
#print(output)

text i am very happy
input_ids tensor([[   3,   23,  183,  182, 1095,    1]])


In [122]:
output['scores'][0][0]

tensor([-20.1368, -12.3730, -13.3437,  ..., -45.1157, -44.9564, -44.8964])

In [132]:
labels = ['joy', 'anger', 'sadness', 'love', 'fear', 'surprise']
ixs = []
for label in labels:
    ixs.append(tokenizer.encode(label)[0])
print(ixs)

[3922, 11213, 24784, 333, 2971, 4158]


In [129]:
output.keys()

odict_keys(['sequences', 'scores', 'encoder_attentions', 'encoder_hidden_states', 'decoder_attentions', 'cross_attentions', 'decoder_hidden_states'])

In [126]:
tokenizer.decode([24784])

'sadness'